In [1]:
import sys

sys.path.append('E:/UADY/CARLA/CARLA_Latest/WindowsNoEditor/PythonAPI/carla')

from abc import ABC, abstractmethod
import carla
import numpy as np
import re
import cv2
import time

In [2]:

class Simulation_parking():

    def __init__(self):
        self.client = carla.Client('localhost', 2000)
        self.world = self.client.get_world()
        self.car_tesla = self.world.get_blueprint_library().filter('*model3*')
        self.exclude_regex = re.compile(
            r'.*(cybertruck|ambulance|fusorosa|vespa|kawasaki|crossbike|harley|micro|carlamotors|yamaha|gazelle|diamondback).*')
        self.vehicle_all = [bp for bp in self.world.get_blueprint_library().filter('*vehicle*') if
                            not self.exclude_regex.match(bp.id)]
        self.available_parking_probability = 0.33
        self.spectator_line = 1

    def load_world(self, world_name):
        self.client.set_timeout(4.0)
        self.client.load_world(world_name)

    def configure_weather(self, weather):
        self.world.set_weather(weather)

    def clean_actors(self):
        for actor in self.world.get_actors().filter('*vehicle*'):
            actor.destroy()
        for sensor in self.world.get_actors().filter('*sensor*'):
            sensor.destroy()

    def get_lights(self):
        return carla.VehicleLightState(carla.VehicleLightState.Position | carla.VehicleLightState.LowBeam)

    def capture_image(self, image, vehicle, image_path):
        image_array = np.frombuffer(image.raw_data, dtype=np.uint8)
        image_array = np.reshape(image_array, (image.height, image.width, 4))
        image_bgr = image_array[:, :, :3]  # Convertir de formato RGBA a formato BGR para OpenCV
        created = time.time()

        image_tags = self.generate_tag_for_image(f'camera_front_mirror', vehicle)

        cv2.imwrite(f'{image_path}_{image_tags}.jpg', image_bgr)

    def generate_tag_for_image(self, sensor_tag, vehicle):
        x = vehicle.get_location().x
        y = vehicle.get_location().y
        yaw = vehicle.get_transform().rotation.yaw
        created = time.time()
        return f's={sensor_tag}_x={x}_y={y}_a={yaw}_t={created}'


In [3]:
class Simulation_parking_perpendicular(Simulation_parking):
    def __init__(self):
        super().__init__()
        self.coordinates_of_parking_trajectories = [
            [406.6, 177.3, 117.5, 40, 30000],
            [651.3, 368.7, 117.5, -140, 30000],
            # [150, -122.5, -140, 30000],

            # [[406.6, 177.3], [636.9, 369.8]],
            # [[651.3, 368.7], [459.2, 206.6]],
            # [[362.7, 369.6], [339.9, 369.6]],
        ]

    def make_images_from_trajectories(self):
        for trajectory in self.coordinates_of_parking_trajectories:
            self.make_images_from_trajectory(trajectory)

    def attach_camera_to_vehicle(self, vehicle):
        ancho_imagen = 1920
        alto_imagen = 1080

        camera_bp = self.world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', str(ancho_imagen))
        camera_bp.set_attribute('image_size_y', str(alto_imagen))
        camera_location_mirror = carla.Location(x=0, y=1, z=1.25)
        camera_Rotation_mirror = carla.Rotation(yaw=45)
        camera_transform = carla.Transform(camera_location_mirror, camera_Rotation_mirror)
        camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
        return camera

    def make_images_from_trajectory(self, trajectory):
        wait = trajectory[4]
        yaw = trajectory[3]
        z_axis = trajectory[2]
        spectator = self.world.get_spectator()

        start_point = carla.Transform(
            carla.Location(x=trajectory[0], y=trajectory[1], z=z_axis),
            carla.Rotation(yaw=yaw)
        )
        spectator.set_transform(start_point)
        vehicle_tesla = self.world.try_spawn_actor(self.car_tesla[0], start_point)
        vehicle_tesla.set_light_state(self.get_lights())
        cv2.waitKey(2000)

        camera = self.attach_camera_to_vehicle(vehicle_tesla)

        camera.listen(
            lambda image: self.capture_image(image, vehicle_tesla, './perpendicular_parking/'))

        vehicle_tesla.set_autopilot(True)

        cv2.waitKey(wait)
        vehicle_tesla.destroy()
        camera.destroy()

In [4]:

class Simulation_parking_angle(Simulation_parking):
    def __init__(self):
        super().__init__()
        self.coordinates_of_parking_trajectories = [
            [245.6, 670.1, 127, -12, 15000],
            [-543.6, 154, 165, 0, 25000],
            # [388.4, 617.0],
        ]

    def make_images_from_trajectories(self):
        for trajectory in self.coordinates_of_parking_trajectories:
            self.make_images_from_trajectory(trajectory)

    def attach_camera_to_vehicle(self, vehicle):
        ancho_imagen = 1920
        alto_imagen = 1080

        camera_bp = self.world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', str(ancho_imagen))
        camera_bp.set_attribute('image_size_y', str(alto_imagen))
        camera_location_mirror = carla.Location(x=0, y=1, z=1.25)
        camera_Rotation_mirror = carla.Rotation(yaw=45)
        camera_transform = carla.Transform(camera_location_mirror, camera_Rotation_mirror)
        camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
        return camera

    def make_images_from_trajectory(self, trajectory):
        wait = trajectory[4]
        yaw = trajectory[3]
        z_axis = trajectory[2]
        spectator = self.world.get_spectator()

        start_point = carla.Transform(
            carla.Location(x=trajectory[0], y=trajectory[1], z=z_axis),
            carla.Rotation(yaw=yaw)
        )
        spectator.set_transform(start_point)
        vehicle_tesla = self.world.try_spawn_actor(self.car_tesla[0], start_point)
        vehicle_tesla.set_light_state(self.get_lights())
        cv2.waitKey(2000)

        camera = self.attach_camera_to_vehicle(vehicle_tesla)

        camera.listen(
            lambda image: self.capture_image(image, vehicle_tesla, './angle_parking/'))

        vehicle_tesla.set_autopilot(True)

        cv2.waitKey(wait)
        vehicle_tesla.destroy()
        camera.destroy()

In [5]:
class Simulation_parking_parallel(Simulation_parking):
    def __init__(self):
        super().__init__()

        self.coordinates_of_parking_trajectories = [
            [879.4, 228.2,132, 140, 18000],
            [79, 279.4, 153, 220, 18000],
            # [716.0, 361.9],
        ]

    def make_images_from_trajectories(self):
        for trajectory in self.coordinates_of_parking_trajectories:
            self.make_images_from_trajectory(trajectory)

    def attach_camera_to_vehicle(self, vehicle):
        ancho_imagen = 1920
        alto_imagen = 1080

        camera_bp = self.world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', str(ancho_imagen))
        camera_bp.set_attribute('image_size_y', str(alto_imagen))
        camera_location_mirror = carla.Location(x=0, y=1, z=1.25)
        camera_Rotation_mirror = carla.Rotation(yaw=45)
        camera_transform = carla.Transform(camera_location_mirror, camera_Rotation_mirror)
        camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
        return camera

    def make_images_from_trajectory(self, trajectory):
        wait = trajectory[4]
        yaw = trajectory[3]
        z_axis = trajectory[2]
        spectator = self.world.get_spectator()

        start_point = carla.Transform(
            carla.Location(x=trajectory[0], y=trajectory[1], z=z_axis),
            carla.Rotation(yaw=yaw)
        )
        spectator.set_transform(start_point)
        vehicle_tesla = self.world.try_spawn_actor(self.car_tesla[0], start_point)
        vehicle_tesla.set_light_state(self.get_lights())
        cv2.waitKey(2000)

        camera = self.attach_camera_to_vehicle(vehicle_tesla)

        camera.listen(
            lambda image: self.capture_image(image, vehicle_tesla, './parallel_parking/'))

        vehicle_tesla.set_autopilot(True)

        cv2.waitKey(wait)
        vehicle_tesla.destroy()
        camera.destroy()


In [6]:
def generate_random_weathers(count_weathers):
    weathers = []
    for i in range(count_weathers):
        weathers.append(
            carla.WeatherParameters(
                # determina la altura del sol. Valores de -90 a 90.
                sun_altitude_angle=np.random.randint(-90, 90),

                # determina la intensidad de la lluvia. Valores de 0 a 100.
                precipitation=np.random.randint(0, 30),

                # determina la creación de charcos en el suelo. Valores de 0 a 100.
                precipitation_deposits=np.random.randint(0, 30),

                # determina la intensidad del viento. Valores de 0 a 100. El viento afecta la dirección de la lluvia y las hojas de los árboles
                wind_intensity=np.random.randint(0, 30),

                # determina la densidad de la niebla. Valores de 0 a 100.
                fog_density=np.random.randint(0, 30),

                # determina la densidad de la tormenta de polvo. Valores de 0 a 100.
                dust_storm=np.random.randint(0, 30),
            )
        )
    return weathers

In [7]:
simulation = Simulation_parking()
simulation.load_world('Town15')

RuntimeError: time-out of 4000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [8]:
weathers = generate_random_weathers(3)

In [9]:
for weather in weathers:
    simulation1 = Simulation_parking_perpendicular()
    simulation1.configure_weather(weather)
    simulation1.make_images_from_trajectories()
    
    simulation2 = Simulation_parking_angle()
    simulation2.configure_weather(weather)
    simulation2.make_images_from_trajectories()
    
    simulation3 = Simulation_parking_parallel()
    simulation3.configure_weather(weather)
    simulation3.make_images_from_trajectories()

In [10]:
esimulation.clean_actors()

In [93]:

# Ejemplo de uso
angle_sim = Simulation_parking_angle()
parallel_sim = Simulation_parking_parallel()
perpendicular_sim = Simulation_parking_perpendicular()

angle_sim.simulate()
parallel_sim.simulate()
perpendicular_sim.simulate()


NameError: name 'Simulation_parking_parallel' is not defined